# Indicadores de agua

## Introducción

Este documento contiene las notas del análisis de los datos disponibles para los municipios que forman parte de alguna de las zonas metropolitanas de acuerdo con el Sistema Urbano Nacional (SUN). El principal objetivo de estas notas es conocer para cuántas zonas metropolitanas se puede calcular una calificación, para que esto pueda hacerse es necesario que todos los municipios de una ciudad cuenten con todas las variables que serán utilizadas para definir los parámetros de la dimensión. La selección de las variables se definirá con el ejercicio descrito en estas notas.


In [2]:
# librerías usadas
%matplotlib inline
from __future__ import division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Datos
La base de datos está en formato delimitado por comas (csv). Este archivo es el resultado de la extracción de datos a los archivos que el INEGI distribuye en sus anuarios estadísticos.

In [3]:
data = pd.read_csv("test.csv")
data.head()

_id   año collection  cve_ent  cve_mun  cve_sun entidad   municipio  \
0  30199  2015    fuentes       30    30199       51     Ver    Zaragoza   
1  30087  2015    fuentes       30    30087       48     Ver      Xalapa   
2  30175  2015    fuentes       30    30175       49     Ver   Tihuatlán   
3  30145  2015    fuentes       30    30145       54     Ver   Soconusco   
4  30138  2015    fuentes       30    30138       50     Ver  Río Blanco   

   total_disponibilidadYacceso  plantas_total        ...          \
0                          NaN            NaN        ...           
1                          NaN            NaN        ...           
2                          NaN            NaN        ...           
3                          NaN            NaN        ...           
4                          NaN            NaN        ...           

   disponen_río_lago_mar  no_disponen  capacidad_total  capacidad_primario  \
0                    NaN          NaN              NaN                 NaN   
1                    NaN          NaN              NaN                 NaN   
2                    NaN          NaN              NaN                 NaN   
3                    NaN          NaN              NaN                 NaN   
4                    NaN          NaN              NaN                 NaN   

   capacidad_secundario  capacidad_terciario  volumen_total  volumen_primario  \
0                   NaN                  NaN            NaN               NaN   
1                   NaN                  NaN            NaN               NaN   
2                   NaN                  NaN            NaN               NaN   
3                   NaN                  NaN            NaN               NaN   
4                   NaN                  NaN            NaN               NaN   

   volumen_secundario  volumen_terciario  
0                 NaN                NaN  
1                 NaN                NaN  
2                 NaN                NaN  
3                 NaN                NaN  
4                 NaN                NaN  

[5 rows x 38 columns]

Cada fila corresponde a un municipio y cada municipio corresponde a una zona metropolitana. Para saber a qué ciudad, o zona metropolitana, corresponde un municipio debe de observarse su clave del SUN (``cve_sun``). El siguiente paso es agrupar a todos los municipios de acuerdo con esta clave.

In [4]:
# Agrupación de todos los municipios según su clave SUN.
zm = data.groupby("cve_sun").count()[[0]]
print "Zonas metropolitanas en total: ",zm.shape[0]

Zonas metropolitanas en total:  59


Después, es necesario filtrar todos los municipios que no tengan ningún ``NaN`` en sus variables.

In [5]:
# Agrupación según clave SUN con todas las variables con valor diferente a "NaN".
zm_sinNaN = data.dropna().groupby("cve_sun").count()[[0]]
print "Zonas metropolitanas sin valores 'NaN': ",zm_sinNaN.shape[0]

Zonas metropolitanas sin valores 'NaN':  38


Tal y como muestran las líneas de código anteriores, exiten 59 zonas metropolitanas; cada una está compuesta por cierto número de municipios. Sin embargo, la tabla de datos original tiene muchos valores vacíos (`NaN`) por lo que, al filtar todos los municipios que no tienen valores vacíos y al agruparlos según su clave SUN, en realidad, de acuerdo con los datos, existen 38 ciudades cuyos municipios cuentan con todas las variables disponibles.

Esto no quiere decir que se cuenta con toda la información para todos los municipios que conforman estas 38 ciudades; la variable ``zm_sinNaN`` enlista el número de municipios por ciudad que no presentan valores ``NaN``. Por ende, el siguiente paso es observar cuántas ciudades tienen toda la información para todos sus municipios.

In [6]:
# Lista de claves SUN de las ciudades sin 'NaNs' para filtrar.
cves_sinNaN = zm_sinNaN.index.tolist()

# Filtro de ciudades con la lista anterior.
zm = zm.loc[zm.index.isin(cves_sinNaN)]

# ¿Qué ciudades tienen la información completa para todos sus municipios?()
(zm_sinNaN / zm)[:10]

_id
cve_sun          
1        1.000000
2        1.000000
3        1.000000
4        0.500000
8        1.000000
9        1.000000
10       0.333333
11       1.000000
12       1.000000
13       0.644737

Aquellas ciudades cuya proporción es igual a uno serán seleccionadas.

In [7]:
# Municipios por ciudad con información completa entre el total de los municipios que existen en esa ciudad.
zm = zm_sinNaN / zm
# Filtar aquellas ciudades cuyos municipios cuentan con toda la información disponible.
zm = zm[zm['_id'] == 1]
zm

_id
cve_sun     
1        1.0
2        1.0
3        1.0
8        1.0
9        1.0
11       1.0
12       1.0
19       1.0
29       1.0
30       1.0
35       1.0
36       1.0
37       1.0
40       1.0
41       1.0
43       1.0
44       1.0
45       1.0
56       1.0

Ya que se tienen las claves SUN de las ciudades sin datos faltantes hay que filtrar los datos de los municipios y declarar una nueva variable con esta información.

In [8]:
muns = data.loc[data['cve_sun'].isin(zm.index.tolist())]
muns.head()

_id   año             collection  cve_ent  cve_mun  cve_sun entidad  \
33  32056  2015  disponibilidadYacceso       32    32056       56     Zac   
53  32017  2015  disponibilidadYacceso       32    32017       56     Zac   
64  32032  2015  disponibilidadYacceso       32    32032       56     Zac   
69  28027  2015  disponibilidadYacceso       28    28027       45   Tamps   
70  28022  2015  disponibilidadYacceso       28    28022       44   Tamps   

       municipio  total_disponibilidadYacceso  plantas_total  \
33     Zacatecas                     146129.0            3.0   
53     Guadalupe                     187910.0            4.0   
64       Morelos                      12324.0            1.0   
69  Nuevo Laredo                     399401.0            5.0   
70     Matamoros                     520318.0            1.0   

          ...          disponen_río_lago_mar  no_disponen  capacidad_total  \
33        ...               0.08493101095749     1.013488              360   
53        ...               0.13442171561134     1.496461            620.8   
64        ...             0.0411929477673422     1.257709               19   
69        ...                              0     1.426136             1617   
70        ...             0.0061454253453729     5.685946              435   

    capacidad_primario  capacidad_secundario  capacidad_terciario  \
33                80.0                     0                280.0   
53                 7.0                    12                601.8   
64                 0.0                     0                 19.0   
69                 0.0                  1617                  0.0   
70                 0.0                   435                  0.0   

    volumen_total  volumen_primario  volumen_secundario  volumen_terciario  
33        9.14544           2.52288                   0            6.62256  
53       15.84684          0.220752            0.378432          15.247656  
64       0.536112                 0                   0           0.536112  
69             31                 0                  31                  0  
70             10                 0                  10                  0  

[5 rows x 38 columns]

## Descripción de variables
La base de datos cuenta con 38 columnas.

In [15]:
for i,a in enumerate(muns.columns):
    print i,a

0 _id
1 año
2 collection
3 cve_ent
4 cve_mun
5 cve_sun
6 entidad
7 municipio
8 total_disponibilidadYacceso
9 plantas_total
10 plantas_primario
11 plantas_secundario
12 plantas_terciario
13 entubada_total
14 entubada_dentro_de_vivienda
15 entubada_fuera_de_vivienda_dentro_de_terreno
16 acarreo_total
17 acarreo_llave_comunitaria
18 acarreo_otra_vivienda
19 acarreo_pipa
20 acarreo_pozo
21 acarreo_rio_lago_arroyo
22 acarreo_lluvia
23 acarreo_no_especificado
24 disponen_total
25 disponen_red_pública
26 disponen_fosa_tanque_(biodigestor)
27 disponen_barranca_grieta
28 disponen_río_lago_mar
29 no_disponen
30 capacidad_total
31 capacidad_primario
32 capacidad_secundario
33 capacidad_terciario
34 volumen_total
35 volumen_primario
36 volumen_secundario
37 volumen_terciario


Las primeras ocho, con índices del cero al siete, se refieren a aspectos como el año, claves geo-estadísticas y demás. A partir de la novena columna, con índice ocho, se encuentran las variables con información relevante sobre consumo, almacenamiento, fuentes de agua, entre otras. Estas últimas variables son a las que se les prestará atención de aquí en adelante, pero antes se almacenarán los nombres de las variables que guardan información del municipio (claves y nombres) pues esto será de utilidad posteriormente.

In [16]:
info_mun = ['entidad','municipio','cve_ent','cve_mun','cve_mun']

### Disponibilidad y acceso al agua
Doce variables corresponden a esta categoría. Se declarará una variable que contenga sólo esta información.
* Total (``total_disponibilidadYacceso``): ¿Está esto en litros o $m^3$?
  * Entubada total (``entubada_total`` %)
       * Dentro de la vivienda (`entubada_dentro_de_vivienda` %)
       * Fuera de la vivienda pero dentro del terreno (`entubada_fuera_de_vivienda_dentro_de_terreno` %)      
  * Por acarreo total (``acarreo_total`` %)
       * De la llave comunitaria (`acarreo_llave_comunitaria` %)
       * De otra vivienda (`acarreo_otra_vivienda` %)
       * De una pipa (`acarreo_pipa` %)
       * De un pozo (`acarreo_pozo` %)
       * De un río, arroyo o lago (`acarreo_río_lago_arroyo` %)
       * De la recolección de lluvia (`acarreo_lluvia` %)
       * No especificado (`acarreo_no_especificado` %)

In [20]:
disponibilidadYacceso = ['total_disponibilidadYacceso','entubada_total','entubada_dentro_de_vivienda',
                         'entubada_fuera_de_vivienda_dentro_de_terreno','acarreo_total','acarreo_llave_comunitaria',
                         'acarreo_otra_vivienda','acarreo_pipa','acarreo_pozo','acarreo_rio_lago_arroyo',
                         'acarreo_lluvia','acarreo_no_especificado']
disponibilidadYacceso += info_mun
disponibilidadYacceso = muns[disponibilidadYacceso]
disponibilidadYacceso.head()

total_disponibilidadYacceso  entubada_total  entubada_dentro_de_vivienda  \
33                     146129.0       97.852582                    94.945136   
53                     187910.0       98.029908                    94.470381   
64                      12324.0       99.050633                    89.538789   
69                     399401.0       98.717830                    95.713960   
70                     520318.0       96.849811                    92.115723   

    entubada_fuera_de_vivienda_dentro_de_terreno  acarreo_total  \
33                                      5.054864       1.855894   
53                                      5.529619       1.925390   
64                                     10.461211       0.933139   
69                                      4.286040       1.040057   
70                                      7.884277       3.110790   

    acarreo_llave_comunitaria  acarreo_otra_vivienda  acarreo_pipa  \
33                   6.342183              46.091445     39.122419   
53                   0.110558              43.946932     39.027087   
64                   0.000000              77.391304      1.739130   
69                  11.434762              40.852191     46.701974   
70                  16.452490              25.250216     46.731743   

    acarreo_pozo  acarreo_rio_lago_arroyo  acarreo_lluvia  \
33      4.756637                 0.368732        1.585546   
53     13.543394                 1.160862        0.138198   
64     20.000000                 0.869565        0.000000   
69      0.096293                 0.000000        0.000000   
70      8.247869                 0.994687        0.988509   

    acarreo_no_especificado entidad     municipio  cve_ent  cve_mun  cve_mun  
33                 1.733038     Zac     Zacatecas       32    32056    32056  
53                 2.072968     Zac     Guadalupe       32    32017    32017  
64                 0.000000     Zac       Morelos       32    32032    32032  
69                 0.914781   Tamps  Nuevo Laredo       28    28027    28027  
70                 1.334487   Tamps     Matamoros       28    28022    28022

### Disponibilidad de drenaje
* Total de ocupantes de viviendas particulares habitadas con disponibilidad de drenaje (``)